<a href="https://colab.research.google.com/github/Davidarr96/Pryecto_MLOps/blob/main/SISTEMA_DE_RECOMENDACION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Machine Learning*** 
Para trabajar con sistemas de recomendación, emplearemos la librería Surprise de la que dejamos aquí la documentación.

Tendremos que llevar nuestro dataset al formato con el que trabaja esta librería.

In [1]:
# Vinculamos google colab con google drive para importar el dataset

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Instalamos surprise

!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3195789 sha256=23094342986cb152357c2765cf5fc51babc940b047ddab4ca3a2aaa3e50bd3dc
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [4]:
# Importamos librerias

import pandas as pd
import sys
import surprise
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

In [5]:
#Creamos nuestra variable reader
reader = Reader()

In [6]:
#Importamos los datasets
data_score = pd.read_csv('/content/drive/MyDrive/plataforma/datasets/score.csv', sep=',', encoding='latin-1')
data_movies = pd.read_csv('/content/drive/MyDrive/plataforma/datasets/plataformas_prom.csv', sep=',', encoding='latin-1')

In [9]:
# Llamamos las columnas a ustulizar de nuestro datasets score.csv
data_score = data_score[ ['userId', 'rating', 'movieId'] ]
data_score.head(3)

,userId,rating,movieId
0,1,1.0,as680
1,1,4.5,ns2186
2,1,5.0,hs2381


In [10]:
# Llamamos las columnas a ustulizar de nuestro datasets plataforma_prom.csv
data_movies = data_movies[ ['Id', 'title'] ]
data_movies.head(3)

,Id,title
0,as1,the grand seduction
1,as2,take care good night
2,as3,secrets of deception


In [11]:
# Unimos ambos dataset, para obtener el nombre de las peliculas con sus votaciones

data_movies_score = pd.merge(left=data_score, right=data_movies, how='left', left_on='movieId', right_on='Id')
data_movies_score.head()

,userId,rating,movieId,Id,title
0,1,1.0,as680,as680,the english civil war
1,1,4.5,ns2186,ns2186,latte and the magic waterstone
2,1,5.0,hs2381,hs2381,la diosa coronada
3,1,5.0,ns3663,ns3663,"frankensteinÃ¢ÂÂs monsterÃ¢ÂÂs monster, fr..."
4,1,5.0,as9500,as9500,kept woman


In [12]:

# Limitamos el dataset a N_filas
N_filas = 100000
data = Dataset.load_from_df( data_movies_score[ ['userId', 'movieId', 'rating']][:N_filas], reader)

In [13]:
# Separamos nuestros datos

trainset, testset = train_test_split (data, test_size=.25)

In [14]:
# Usaremos un modelo de Singular Value Decomposition

from surprise import SVD
model = SVD()

In [15]:
# Entrenamos el modelo

model.fit(trainset)

In [16]:
# Predecimos

predictions = model.test(testset)

In [17]:
predictions[1]


Prediction(uid=804, iid='ns5661', r_ui=3.5, est=3.479940121578747, details={'was_impossible': False})

In [18]:
# Hacemos una predicción al azar para usuario y película

model.predict(1, 'as680')

Prediction(uid=1, iid='as680', r_ui=None, est=4.315691357088211, details={'was_impossible': False})

In [28]:
# Tomaremos un usuario para hacerle una recomendación, y nos basaremos en peliculas que haya calificado con 4 o 5 estrellas

usuario = 878
rating = 4   
df_user = data_movies_score[(data_score['userId'] == usuario) & (data_score['rating'] >= rating)]
df_user = df_user.reset_index(drop=True)

In [29]:
#df_user['Name'] = data_movies['title'].loc[df_user.userId].values
df_user

,userId,rating,movieId,Id,title
0,878,4.0,hs2478,hs2478,night of the living dead
1,878,4.5,ns6138,ns6138,american crime
2,878,4.0,as9375,as9375,prometheus (4k uhd)
3,878,4.0,as6217,as6217,sincerity
4,878,4.0,ds1095,ds1095,mr. duck steps out
...,...,...,...,...,...
211,878,4.0,ns5970,ns5970,Ã¢ÂÂmaj rati Ã¢ÂÂÃ¢ÂÂketeki
212,878,4.0,as4203,as4203,dark was the night
213,878,4.5,ns2927,ns2927,we bare bears
214,878,4.0,ns911,ns911,monster


In [30]:
recomendaciones_usuario = data_movies.iloc[:4499].copy()
print(recomendaciones_usuario.shape)
recomendaciones_usuario.head()


(4499, 2)


,Id,title
0,as1,the grand seduction
1,as2,take care good night
2,as3,secrets of deception
3,as4,pink: staying true
4,as5,monster maker


In [31]:
# Debemos extraer las películas que ya ha visto

usuario_vistas = data_score[ data_score['userId'] == usuario]
print(usuario_vistas.shape)
usuario_vistas.head()

(552, 3)


,userId,rating,movieId
87221,878,4.0,hs2478
87222,878,2.5,ns7561
87223,878,4.5,ns6138
87224,878,4.0,as9375
87225,878,4.0,as6217


In [32]:
# Recomendamos

recomendaciones_usuario['Estimate_Score'] = recomendaciones_usuario['Id'].apply(lambda x: model.predict(usuario, x).est)
     

recomendaciones_usuario = recomendaciones_usuario.sort_values('Estimate_Score', ascending=False)
recomendaciones_usuario.head()

,Id,title,Estimate_Score
3429,as3430,a dangerous man,4.250809
121,as122,where is chicky? - chapter four,4.092012
1069,as1070,secret life of the hospital bed,4.047677
2775,as2776,counting cars,4.041508
2910,as2911,card sharks with jim perry,4.027356


In [33]:
# Evaluación

from surprise import accuracy

accuracy.rmse(predictions)
     


RMSE: 0.9548


0.9548243166257058